In [1]:
# libraries

import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import time
from pathlib import Path

In [2]:
image_path = Path("../02/mats/")
office_path = Path(image_path / "office/office/input/")
pedestrian_path = Path(image_path / "pedestrian/pedestrian/input/")
# print(os.listdir(office_path))

### 1. pedestrian dataset, median and mean buffor

I. median buffor

In [4]:
N = 60
BUF = np.zeros ((240,360,N),np.uint8)
iN = 0
path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2

for i in range(1, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)

    BUF[:,:,iN] = gray
    iN += 1
    if iN == 60:
        iN = 0


    cv2.waitKey(10)

    background = np.median(BUF, axis=2).astype(np.uint8)

    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

    I_VIS = I.copy()
    if(stats.shape[0]>1 and i > 60):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.944056486422795, recall = 0.9460460472429647, F1 = 0.9450502197065502


II. mean buffor


In [16]:
N = 60
BUF = np.zeros ((240,360,N),np.uint8)
iN = 0
path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2

for i in range(1, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)

    BUF[:,:,iN] = gray
    iN += 1
    if iN == 60:
        iN = 0


    cv2.waitKey(10)

    background = np.mean(BUF, axis=2).astype(np.uint8)

    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


    if(stats.shape[0]>1 and i > 30):
        I_VIS = I.copy()
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


        TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
        TP_S = np . sum ( TP_M )
        TP = TP + TP_S

        FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
        FP_S = np . sum ( FP_M )
        FP = FP + FP_S

        FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
        FN_S = np . sum ( FN_M )
        FN = FN + FN_S

        TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
        TN_S = np . sum ( TN_M )
        TN = TN + TN_S

    cv2.imshow('I_VIS', diff)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.7695364820060163, recall = 0.9411369800477993, F1 = 0.8467299654514266


Median seems to work better for moving objects. Lrt's test it on office dataset

In [6]:
N = 60
BUF = np.zeros ((240,360,N),np.uint8)
iN = 0
path = '../02/mats/office/office/input'
gt_path = '../02/mats/office/office/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2

for i in range(570, 2050,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)

    BUF[:,:,iN] = gray
    iN += 1
    if iN == 60:
        iN = 0


    cv2.waitKey(10)

    background = np.median(BUF, axis=2).astype(np.uint8)

    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


    if(stats.shape[0]>1 and i > 60):
        I_VIS = I.copy()
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.3253340895943887, recall = 0.44478569785886735, F1 = 0.37579595391529286


and for highway

In [7]:
N = 60
BUF = np.zeros ((240,320,N),np.uint8)
iN = 0
path = '../02/mats/highway/highway/input'
gt_path = '../02/mats/highway/highway/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2

for i in range(470, 1700,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    # cv2.imshow('gray', gray)

    BUF[:,:,iN] = gray
    iN += 1
    if iN == 60:
        iN = 0


    cv2.waitKey(10)

    background = np.median(BUF, axis=2).astype(np.uint8)

    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


    if(stats.shape[0]>1 and i > 30):
        I_VIS = I.copy()
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.5054557413312696, recall = 0.9081555561434917, F1 = 0.6494464789502231


### 2. aproksymacja mediany i średniej

I. aproksymacja średniej

In [11]:
N = 60

path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2
alpha = 0.1

background = cv2.cvtColor(cv2.imread ( f'{path}/in000001.jpg '), cv2.COLOR_BGR2GRAY).astype(np.uint8)
for i in range(300, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(10)

    #BGN = αIN +(1 −α)BGN−1
    background =  (alpha * gray + (1 - alpha) * background).astype(np.uint8)
    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


    if(stats.shape[0]>1 and i > 60):
        I_VIS = I.copy()
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.33947426629588506, recall = 0.9202544031311155, F1 = 0.4959840890984761


median appx

In [9]:
N = 60

path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2
alpha = 0.01

background = cv2.cvtColor(cv2.imread ( f'{path}/in000001.jpg '), cv2.COLOR_BGR2GRAY).astype(np.uint8)
for i in range(300, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(10)

# if BGN−1 <IN then BGN = BGN−1 +1
# if BGN−1 >IN then BGN = BGN−1 −1
# otherwise BGN = BGN−1    
    background = np.where(background < gray, background + 1, background - 1).astype(np.uint8)
    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)


    if(stats.shape[0]>1 and i > 60):
        I_VIS = I.copy()
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)



precision = TP / (TP + FP)
recall = TP / (TP + FN)
F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.9670960050635712, recall = 0.940150542130888, F1 = 0.9534329316685144


### konserwatywne podejście do aktualizacji

aktualizujemy tylko te piksele maski gdzie jasność uległa zmianie

In [56]:


path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2
alpha = 0.005
treshold = 20

I = cv2.imread ( f'{path}/in000001.jpg ')
gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
background = cv2.cvtColor(cv2.imread ( f'{path}/in000001.jpg '), cv2.COLOR_BGR2GRAY).astype(np.uint8)
dilated = np.zeros_like(gray)
for i in range(300, 1100,step):
    I_old = gray
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(10)

    frame_diff = cv2.absdiff(gray, I_old)

    #constervative update, only update the background on the pixels that are not background, so 255 on binarized image
    ### TODO
    # tam gdzie dilated wykrył tło, tam bacground będzie aktualizowane, zgodnie z algorytmem aproksymacji mediany
    # background = np.where(dilated > treshold, np.where(background < gray, background + 1, background - 1), background).astype(np.uint8)
    background = np.where(dilated > treshold, (alpha * gray + (1 - alpha) * background), background).astype(np.uint8)
    test = np.where(dilated > treshold, 255, 0).astype(np.uint8)
    diff = cv2.absdiff(gray, background)
    binary = cv2.threshold(diff, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 7)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

    I_VIS = I.copy()
    if(stats.shape[0]>1 and i > 60):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', dilated)


if TP + FP == 0:
    precision = 0
else:
    precision = TP / (TP + FP)

if TP + FN == 0:
    recall = 0
else:
    recall = TP / (TP + FN)

if precision + recall == 0:
    F1 = 0
else:
    F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.7678923955673712, recall = 0.9585030884478907, F1 = 0.8526751136247388


### Opencv methods

In [14]:
BackgroundSubscractor = cv2.createBackgroundSubtractorMOG2(history=60, varThreshold=30, detectShadows=True)

path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 2


gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
for i in range(300, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(10)

    fgmask = BackgroundSubscractor.apply(I)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, (7,7))
    
    apply = cv2.cvtColor(cv2.bitwise_and(I, I, mask=fgmask), cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(apply, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (5,5))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

    I_VIS = I.copy()
    if(stats.shape[0]>1 and i > 60):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)


if TP + FP == 0:
    precision = 0
else:
    precision = TP / (TP + FP)

if TP + FN == 0:
    recall = 0
else:
    recall = TP / (TP + FN)

if precision + recall == 0:
    F1 = 0
else:
    F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.6387540249926179, recall = 0.9498163027569522, F1 = 0.7638301884299745


KNN

In [16]:
BackgroundSubscractor = cv2.createBackgroundSubtractorKNN(history=60, dist2Threshold=500, detectShadows=False)

path = '../02/mats/pedestrian/pedestrian/input'
gt_path = '../02/mats/pedestrian/pedestrian/groundtruth'
TP , TN , FP , FN = 0 , 0 , 0 , 0
step = 3


gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
for i in range(300, 1100,step):
    I=cv2.imread ( f'{path}/in%06d.jpg ' % i )
    I_GT=cv2.imread ( f'{gt_path}/gt%06d.png ' % i )
    gray = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY)
    cv2.waitKey(10)

    fgmask = BackgroundSubscractor.apply(I)
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, (7,7))
    
    apply = cv2.cvtColor(cv2.bitwise_and(I, I, mask=fgmask), cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(apply, 20, 255, cv2.THRESH_BINARY)[1]
    median = cv2.medianBlur(binary, 3)
    dilated = cv2.morphologyEx(median, cv2.MORPH_OPEN, (7,7))

    retval , labels , stats , centroids = cv2.connectedComponentsWithStats(dilated)

    I_VIS = I.copy()
    if(stats.shape[0]>1 and i > 60):
        tab = stats[1:,4]
        pi = np.argmax(tab)
        # print(f'max area = {tab[pi]}')

        if (tab[pi]>400):
            pi = pi + 1
            cv2.rectangle(I_VIS, (stats[pi,0],stats[pi,1]), (stats[pi,0]+stats[pi,2],stats[pi,1]+stats[pi,3]), (255,0,0), 2)
            cv2.putText(I_VIS, "%f" %stats[pi,4],(stats[pi,0],stats[pi,1]),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0))


            TP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 255))
            TP_S = np . sum ( TP_M )
            TP = TP + TP_S

            FP_M = np.logical_and((dilated == 255), (I_GT[:,:,0] == 0))
            FP_S = np . sum ( FP_M )
            FP = FP + FP_S

            FN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 255))
            FN_S = np . sum ( FN_M )
            FN = FN + FN_S

            TN_M = np.logical_and((dilated == 0), (I_GT[:,:,0] == 0))
            TN_S = np . sum ( TN_M )
            TN = TN + TN_S

    cv2.imshow('I_VIS', I_VIS)


if TP + FP == 0:
    precision = 0
else:
    precision = TP / (TP + FP)

if TP + FN == 0:
    recall = 0
else:
    recall = TP / (TP + FN)

if precision + recall == 0:
    F1 = 0
else:
    F1 = 2 * (precision * recall) / (precision + recall)

print(f'precision = {precision}, recall = {recall}, F1 = {F1}')

cv2.destroyAllWindows()

precision = 0.360016940588571, recall = 0.9875174365667793, F1 = 0.5276644697423607
